In [1]:
from npearth._cholesky_update import cholupdate, cholsolve, choldowndate

import numpy as np

In [191]:
A = np.array([1, 2, 1, 2, 3, 2, 1, 2, 1]).reshape((3, 3))
V = A.T @ A
V_n = np.array([ 6., 10.,  7., 10., 17., 14.,  7., 14., 31.]).reshape((3, 3))

x_u = np.array([1, 4, 25]) / np.sqrt(25)
x_d = np.array([1, 4, 0]) / np.sqrt(25)

np.outer(x_u, x_u)

array([[ 0.04,  0.16,  1.  ],
       [ 0.16,  0.64,  4.  ],
       [ 1.  ,  4.  , 25.  ]])

In [ ]:
def get_cholupdate_vector(V_new: np.ndarray, V_old: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Returns the vector for updating SP"""
    dVM = V_new[:,-1] - V_old[:,-1]
    return dVM / np.sqrt(dVM[-1])

get_cholupdate_vector(V_n, V)

array([0.2, 0.8, 5. ])

In [162]:
np.linalg.eig(V)

EigResult(eigenvalues=array([2.88614066e+01, 1.02779722e-16, 1.38593384e-01]), eigenvectors=array([[-4.54401349e-01, -7.07106781e-01,  5.41774320e-01],
       [-7.66184591e-01,  5.15817853e-16, -6.42620551e-01],
       [-4.54401349e-01,  7.07106781e-01,  5.41774320e-01]]))

In [163]:
np.outer(x_d, x_d)

array([[0.04, 0.16, 0.  ],
       [0.16, 0.64, 0.  ],
       [0.  , 0.  , 0.  ]])

In [183]:
V_n = V + np.outer(x_u, x_u) - np.outer(x_d, x_d)
L = np.linalg.cholesky(V + 1e-10 * np.eye(3))
L_n = np.linalg.cholesky(V_n + 1e-10 * np.eye(3))
L_n

array([[2.44948974, 0.        , 0.        ],
       [4.0824829 , 0.57735027, 0.        ],
       [2.85773803, 4.04145188, 2.54950976]])

In [179]:
from copy import deepcopy
Lc = choldowndate(cholupdate(deepcopy(L), deepcopy(x_u)), deepcopy(x_d))

In [186]:
Lc @ Lc.T

array([[ 6., 10.,  7.],
       [10., 17., 14.],
       [ 7., 14., 31.]])

In [178]:
L_n @ L_n.T

array([[ 6., 10.,  7.],
       [10., 17., 14.],
       [ 7., 14., 31.]])

In [86]:
n = 100
B = np.random.randn(n, n)
A = B.T @ B + np.eye(n) * 1e-1  # small ridge term improves SPD-ness
b = np.random.randn(n)

cond_A = np.linalg.cond(A)
print(f"Condition number (good): {cond_A:.2e}")

Condition number (good): 3.94e+03


In [100]:
U, _ = np.linalg.qr(np.random.randn(n, n))
V, _ = np.linalg.qr(np.random.randn(n, n))
sing_vals = np.logspace(0, -8, n)  # from 1 to 1e-10
B_bad = U @ np.diag(sing_vals) @ V.T
A_bad = B_bad.T @ B_bad
b_bad = np.random.randn(n)

cond_A_bad = np.linalg.cond(A_bad)
print(f"Condition number (bad): {cond_A_bad:.2e}")

Condition number (bad): 1.06e+16


In [121]:
x_true = np.random.randn(n)

b_bad = A_bad @ x_true
x_est = np.linalg.solve(A_bad, b_bad)

alpha = 1e-8
A_bad_fix = A_bad + np.eye(n) * alpha * A_bad.diagonal().mean() # small ridge term
x_est_fix = np.linalg.solve(A_bad_fix, b_bad)

b_good = A @ x_true
x_est_good = np.linalg.solve(A, b_good)

print("Relative error (bad):", np.linalg.norm(x_true - x_est) / np.linalg.norm(x_true))
print("Relative error (good):", np.linalg.norm(x_true - x_est_good) / np.linalg.norm(x_true))
print("Relative error (bad + ridge):", np.linalg.norm(x_true - x_est_fix) / np.linalg.norm(x_true))

Relative error (bad): 0.06854079470444051
Relative error (good): 9.019870023301938e-14
Relative error (bad + ridge): 0.6122697248697435


In [22]:
x_true = np.random.randn(n)

b_bad = A_bad @ x_true
A_bad_inv = np.linalg.inv(A_bad)
x_est = A_bad_inv @ b_bad

A_bad_fix = A_bad + np.eye(n) * 1e-5  # small ridge term
A_bad_fix_inv = np.linalg.inv(A_bad_fix)
x_est_fix = A_bad_fix_inv @ b_bad

b_good = A @ x_true
A_good_inv = np.linalg.inv(A)
x_est_good = A_good_inv @ b_good

print("Relative error (bad):", np.linalg.norm(x_true - x_est) / np.linalg.norm(x_true))
print("Relative error (good):", np.linalg.norm(x_true - x_est_good) / np.linalg.norm(x_true))
print("Relative error (bad + ridge):", np.linalg.norm(x_true - x_est_fix) / np.linalg.norm(x_true))

Relative error (bad): 50.53493753465121
Relative error (good): 7.850387000140891e-14
Relative error (bad + ridge): 0.8007842867939184


In [96]:
x_true = np.random.randn(n)

# b_bad = A_bad @ x_true
# L_bad = np.linalg.cholesky(A_bad)
# x_est = cholsolve(L_bad, b_bad)

A_bad_fix = A_bad + np.eye(n) * 1e-6  # small ridge term
L_bad_fix = np.linalg.cholesky(A_bad_fix)
x_est_fix = cholsolve(L_bad_fix, b_bad)

b_good = A @ x_true
L_good = np.linalg.cholesky(A)
x_est_good = cholsolve(L_good, b_good)

print("Relative error (bad):", np.linalg.norm(x_true - x_est) / np.linalg.norm(x_true))
print("Relative error (good):", np.linalg.norm(x_true - x_est_good) / np.linalg.norm(x_true))
print("Relative error (bad + ridge):", np.linalg.norm(x_true - x_est_fix) / np.linalg.norm(x_true))

Relative error (bad): 1.7453130740230705
Relative error (good): 2.3599002845838348e-14
Relative error (bad + ridge): 1.2028693657579497


Do an experiment. See if I can do many cholesky updates, solve system, and then test error compared to full "true" solve. Afterwards add diagonal as $\epsilon D$ and test again for numerical stability:

$$Ax = LL^Tx = b,$$

Then we update L, as $L' = L + aa^T$